In [ ]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, Normalize
from tempfile import NamedTemporaryFile
import seaborn as sns
import numpy as np
import six
import tensorflow as tf

plt.style.use('seaborn-talk')
sns.set_context("talk", font_scale=1.4)
sess = ed.get_session()

sns.palplot(sns.color_palette())

In [ ]:
# this can be done only before using Edward
ed.set_seed(42)

# Coin

In [ ]:
from edward.models import Bernoulli, Beta, Empirical, Uniform

In [ ]:
N = 100  # number of coin flip observations in dataset

In [ ]:
def build_fair_dataset(N):
    pheads = tf.constant(0.5)
    c = Bernoulli(probs=pheads, sample_shape=N)
    return sess.run([pheads, c])

In [ ]:
def build_unfair_dataset(N):
    pheads = tf.constant(0.05)
    c = Bernoulli(probs=pheads, sample_shape=N)
    return sess.run([pheads, c])

In [ ]:
def build_dataset(N):
    pheads = Uniform(low=0.0, high=1.0)
    c = Bernoulli(probs=pheads, sample_shape=N)
    return sess.run([pheads, c])

In [ ]:
x = tf.range(-0.2, 1.2, 0.001)
plt.plot(*sess.run([x, Uniform(low=0.0, high=1.0).prob(x)]));
#plt.plot(*sess.run([x, Beta(concentration1=1.0, concentration0=1.0).prob(x)]));
plt.ylim((-0.2, 1.2));

In [ ]:
# DATA
pheads_true, c_train = build_dataset(N)

In [ ]:
pheads_true

In [ ]:
c_train

In [ ]:
sum(c_train == 0)

In [ ]:
sum(c_train == 1)

In [ ]:
pheads_fair = Beta(concentration1=1000.0, concentration0=1000.0)  # blue
pheads_unfair = Beta(concentration1=0.1, concentration0=0.1)  # green
# pheads_unknown = Uniform(low=0.0, high=1.0)  # red
pheads_unknown = Beta(concentration1=1.0, concentration0=1.0)  # red

In [ ]:
x = tf.range(0.0, 1.0, 0.001)
plt.plot(*sess.run([x, pheads_fair.prob(x)]));
plt.plot(*sess.run([x, pheads_unfair.prob(x)]));
plt.plot(*sess.run([x, pheads_unknown.prob(x)]));
plt.axvline(x=pheads_true);

In [ ]:
# FORWARD MODEL
pheads = pheads_unknown
c = Bernoulli(probs=pheads, sample_shape=N)

#### Variational Inference (VI)

In [ ]:
# BACKWARD MODEL
q_pheads_concentration1 = tf.nn.softplus(tf.Variable(tf.random_normal([])))
q_pheads_concentration0 = tf.nn.softplus(tf.Variable(tf.random_normal([])))
q_pheads = Beta(concentration1=q_pheads_concentration1,
                concentration0=q_pheads_concentration0)

In [ ]:
x = tf.range(-5.0, 5.0, 0.001)
plt.plot(*sess.run([x, tf.nn.softplus(x)]));

In [ ]:
# INFERENCE
inference = ed.KLqp(latent_vars={pheads: q_pheads},
                    data={c: c_train})
inference.run(n_samples=20, n_iter=1000)
#inference.run(n_samples=1000, n_iter=1000)

In [ ]:
# T = 10000  # number of empirical samples
# q_pheads_samples = sess.run(q_pheads.sample(sample_shape=T))

In [ ]:
# plt.plot(q_pheads_samples);
# plt.axhline(y=pheads_true);

In [ ]:
sess.run({key: val for
          key, val in six.iteritems(q_pheads.parameters)
          if isinstance(val, tf.Tensor)})

In [ ]:
plt.plot(*sess.run([x, pheads.prob(x)]));  # blue
plt.plot(*sess.run([x, pheads_cond.prob(x)],  # green
                   {c: c_train}));
plt.plot(*sess.run([x, q_pheads.prob(x)]));  # red
# plt.hist(q_pheads_samples,  # red
#          bins=100, range=(0.0, 1.0),
#          normed=True);
plt.axvline(x=pheads_true);  # blue

In [ ]:
# CRITICISM
mean, stddev = sess.run([q_pheads.mean(), q_pheads.stddev()])
print("Inferred posterior mean:")
print(mean)
print("Inferred posterior std:")
print(stddev)